In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

DATA_DIR = Path("data")
CLEAN_DIR = DATA_DIR / "clean"

results_path = CLEAN_DIR / "results_all.csv"
startlists_path = CLEAN_DIR / "startlists_all.csv"

print("Loading:")
print(" -", results_path)
print(" -", startlists_path)

results = pd.read_csv(results_path, parse_dates=["race_date"])
startlists = pd.read_csv(startlists_path, parse_dates=["race_date"])

results.head(), startlists.head()


Loading:
 - data/clean/results_all.csv
 - data/clean/startlists_all.csv


(  Category Name  Place  RacerID First Name     Last Name  \
 0     Men Elite      1   129684       Toon    Vandebosch   
 1     Men Elite      2   174032     Victor  Van de putte   
 2     Men Elite      3   144667      Niels    Vandeputte   
 3     Men Elite      4   181210       Wout       Janssen   
 4     Men Elite      5   153653    Cameron         Mason   
 
                 Team Name     Time  License  Carried Points  Scored Points  \
 0         CRELAN-CORENDON  1:04:00      NaN      151.572152     135.875229   
 1  DESCHACHT-HENS CX TEAM  1:04:01      NaN      170.054789     140.548828   
 2                     NaN  1:04:01      NaN      130.398560     145.222426   
 3                     NaN  1:04:02      NaN      181.522616     149.896025   
 4            SEVEN RACING  1:04:02      NaN      157.932693     154.569623   
 
    series_name                   race_name  race_date race_location  \
 0  Exact Cross  Robotland Cyclocross Essen 2025-10-18    Essen, BEL   
 1  Exact Cr

In [13]:
def normalize_name(s):
    if pd.isna(s):
        return None
    s = str(s).strip().lower()
    # remove accents
    s = (
        s.replace("é", "e").replace("è", "e").replace("ë", "e")
         .replace("ó", "o").replace("ò", "o").replace("ö", "o")
         .replace("á", "a").replace("à", "a").replace("ä", "a")
         .replace("ü", "u")
    )
    # collapse multiple spaces
    s = re.sub(r"\s+", " ", s)
    return s

results["rider_name_norm"] = results["rider_name"].apply(normalize_name)
startlists["rider_name_norm"] = startlists["rider_name"].apply(normalize_name)


In [14]:
# Ensure string dtype for merge compatibility
startlists["UCI ID"] = startlists["UCI ID"].astype(str)


In [15]:
results = results.sort_values(["rider_name_norm", "race_date"])

def add_form_features(df):
    df = df.copy()

    # Races so far per rider
    df["races_so_far"] = df.groupby("rider_name_norm").cumcount()

    place_shifted = df.groupby("rider_name_norm")["Place"].shift(1)

    df["avg_place_last3"] = (
        place_shifted.groupby(df["rider_name_norm"])
        .rolling(3).mean().reset_index(level=0, drop=True)
    )

    df["best_place_last5"] = (
        place_shifted.groupby(df["rider_name_norm"])
        .rolling(5).min().reset_index(level=0, drop=True)
    )

    df["last_place"] = place_shifted
    df["days_since_last_race"] = (
        df.groupby("rider_name_norm")["race_date"].diff().dt.days
    )

    df["last_carried_points"] = df.groupby("rider_name_norm")["Carried Points"].shift(1)
    df["last_scored_points"] = df.groupby("rider_name_norm")["Scored Points"].shift(1)

    return df

results_feat = add_form_features(results)
results_feat.head()


,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points,...,race_id,rider_name,rider_name_norm,races_so_far,avg_place_last3,best_place_last5,last_place,days_since_last_race,last_carried_points,last_scored_points
1440,Women Elite,18,196998,Adèle,Hurteloup,VELOPRO-EGS GROUP-ALPHAMOTORHOMES,0:54:03,NaN,392.521707,343.504268,...,20251101_x-o-badkamers-trofee_koppenbergcross_...,Adèle Hurteloup,adele hurteloup,0,NaN,NaN,NaN,NaN,NaN,NaN
306,Men Junior,20,240497,Adne,Cappon,NaN,41:22,NaN,370.000000,305.302491,...,20251025_exact-cross_internationale-cyclocross...,Adne Cappon,adne cappon,0,NaN,NaN,NaN,NaN,NaN,NaN
1138,Men Junior,44,241251,Adriaan,Van bommel,NaN,-2 LAPS,NaN,370.000000,455.701145,...,20251102_x-o-badkamers-trofee_rapencross_loker...,Adriaan Van bommel,adriaan van bommel,0,NaN,NaN,NaN,NaN,NaN,NaN
1016,Women Junior,20,229563,Aitana,Gutierrez velarde,NaN,0:41:11,NaN,449.740608,456.297097,...,20251109_uec-cyclocross_european-championships...,Aitana Gutierrez velarde,aitana gutierrez velarde,0,NaN,NaN,NaN,NaN,NaN,NaN
198,Women Elite,18,202425,Alana,Polfliet,NaN,0:56:06,NaN,505.277027,393.450092,...,20251004_exact-cross_berencross_tielt-meulebek...,Alana Polfliet,alana polfliet,0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
out_path = CLEAN_DIR / "results_with_features.csv"
results_feat.to_csv(out_path, index=False)
out_path


PosixPath('data/clean/results_with_features.csv')

In [17]:
feature_cols = [
    "races_so_far",
    "avg_place_last3",
    "best_place_last5",
    "last_place",
    "days_since_last_race",
    "last_carried_points",
    "last_scored_points",
]

train_base = results_feat[
    ["race_id", "rider_name", "rider_name_norm", "Place"] + feature_cols
].copy()

train_base.head()


,race_id,rider_name,rider_name_norm,Place,races_so_far,avg_place_last3,best_place_last5,last_place,days_since_last_race,last_carried_points,last_scored_points
1440,20251101_x-o-badkamers-trofee_koppenbergcross_...,Adèle Hurteloup,adele hurteloup,18,0,NaN,NaN,NaN,NaN,NaN,NaN
306,20251025_exact-cross_internationale-cyclocross...,Adne Cappon,adne cappon,20,0,NaN,NaN,NaN,NaN,NaN,NaN
1138,20251102_x-o-badkamers-trofee_rapencross_loker...,Adriaan Van bommel,adriaan van bommel,44,0,NaN,NaN,NaN,NaN,NaN,NaN
1016,20251109_uec-cyclocross_european-championships...,Aitana Gutierrez velarde,aitana gutierrez velarde,20,0,NaN,NaN,NaN,NaN,NaN,NaN
198,20251004_exact-cross_berencross_tielt-meulebek...,Alana Polfliet,alana polfliet,18,0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
train_path = CLEAN_DIR / "train_race_rows.csv"
train_base.to_csv(train_path, index=False)
train_path


PosixPath('data/clean/train_race_rows.csv')

In [19]:
# Prepare startlists merge keys
startlists["UCI ID"] = startlists["UCI ID"].replace("None", np.nan)
startlists["merge_id"] = startlists["UCI ID"].copy()

# If UCI ID missing → fallback to normalized name
startlists.loc[startlists["merge_id"].isna(), "merge_id"] = (
    "name:" + startlists["rider_name_norm"]
)

# Build the same kind of merge key for results
results_feat["merge_id"] = np.where(
    results_feat.get("UCI ID", pd.Series([None]*len(results_feat))).notna(),
    results_feat.get("UCI ID", None),
    "name:" + results_feat["rider_name_norm"]
)

# Deduplicate on merge_id + race_id on the results side
results_dedup = results_feat.drop_duplicates(subset=["merge_id", "race_id"])


In [20]:
merged = startlists.merge(
    results_dedup[
        ["merge_id", "race_id"] + feature_cols + ["Place"]
    ],
    on="merge_id",
    how="left",
)

merged.head(20)


,Nr,Naam,UCI ID,Nat,Club,UCI Rank,series_name,race_name,race_date,race_location,...,merge_id,race_id_y,races_so_far,avg_place_last3,best_place_last5,last_place,days_since_last_race,last_carried_points,last_scored_points,Place
0,1.0,AERTS Toon,10007586087.0,BEL,DESCHACHT-HENS CX TEAM,2.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10007586087.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NYS Thibau,10065004835.0,BEL,BALOISE GLOWI LIONS,9.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10065004835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,VANTHOURENHOUT Michael,10007155651.0,BEL,PAUWELS SAUZEN - ALTEZ INDUSTRIEBOUW CT,1.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10007155651.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,VANDEPUTTE Niels,10016327609.0,BEL,ALPECIN-DECEUNINCK DEVELOPMENT TEAM,4.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10016327609.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,WYSEURE Joran,10064921777.0,BEL,CRELAN-CORENDON,5.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10064921777.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,SWEECK Laurens,10006912646.0,BEL,CRELAN-CORENDON,7.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10006912646.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.0,VAN DER HAAR Lars,10006118660.0,NED,BALOISE GLOWI LIONS,8.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10006118660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,RONHAAR Pim,10023108212.0,NED,BALOISE GLOWI LIONS,10.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10023108212.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,ORTS LLORET Felipe,10009004715.0,ESP,RIDLEY RACING TEAM,11.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10009004715.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,KUHN Kevin,10009746763.0,SUI,HEIZOMAT - CUBE,14.0,NaN,Telenet Superprestige AARDBEIENCROSS-MERKSPLAS...,NaT,NaN,...,10009746763.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
out_path = CLEAN_DIR / "startlists_enriched.csv"
merged.to_csv(out_path, index=False)
out_path


PosixPath('data/clean/startlists_enriched.csv')